# Python HANA ML API

<div class="alert alert-block alert-info">
<b>Time series forecasting of monthly ozone rate.</b> <br>
</div>

### Create an HANA Dataframe for the actual series

In [ ]:
# Connect using the HANA secure user store
from hana_ml import dataframe as hd
conn = hd.ConnectionContext(userkey='MLDB07_KEY')
# Get Series Data
sql_cmd = 'SELECT * FROM "APL_SAMPLES"."OZONE_RATE_LA" ORDER BY "Date"'
series_data = hd.DataFrame(conn, sql_cmd)

### Put the series in a Pandas Dataframe and show individual values

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
df = series_data.collect()
df['Date'] = pd.to_datetime(df['Date'])
df['OzoneRateLA'] = pd.to_numeric(df['OzoneRateLA'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
del df['Date']
pd.pivot_table(df, index='Month', columns=['Year']).T

### Forecast with APL

#### Running the forecast

In [ ]:
# Specify the model  
from hana_ml.algorithms.apl.time_series import AutoTimeSeries
model = AutoTimeSeries(
    conn_context=conn, 
    time_column_name= 'Date',
    target= 'OzoneRateLA',
    horizon= 12
    )
# Run the model 
output_data = model.fit_predict(series_data)

##### Plotting the predicted values

In [ ]:
# Make the output a Pandas Dataframe
df = output_data.collect()
# Build a line chart
import hvplot.pandas
df.hvplot.line(
 'Date' , ['ACTUAL','PREDICTED'], 
 value_label='Ozone Rate', 
 title = 'Monthly Ozone Rate in LA',
 fontsize={'title': 10, 'labels': 10},
 legend = 'bottom', height = 350, width = 900
) *\
df.hvplot.area(
 'Date' , 'LOWER_INT_95PCT', 'UPPER_INT_95PCT', 
 line_color = 'white', color = 'orange', alpha=0.2
)

##### Components found

In [ ]:
d = model.get_model_components()
df = pd.DataFrame(list(d.items()), columns=["Component", "Value"])
df.style.hide_index()

##### Performance Indicators

In [ ]:
d = model.get_performance_metrics()
# Average each indicator across the horizon time window
apm = []
for k, v in d.items():
   apm.append((k, round(np.mean(v),4)))
df = pd.DataFrame(apm, columns=["Indicator", "Value"])
df.drop(pd.Index(np.where(df['Indicator'].isin(['L1','L2','U2','P2']))[0]), inplace = True)
df.style.hide_index()